In [ ]:

!pip install gym[atari] autorom[accept-rom-license]
!pip install pybullet
!pip install ale-py
!pip install -U ipywidgets
!pip install gymnasium[atari,accept-rom-license]
!pip install gputil
!pip install gym[classic_control]
!sudo apt-get install xvfb
!pip install pyvirtualdisplay Pillow
!pip install huggingface_sb3
!pip install stable-baselines3[extra]
!pip install --user --upgrade git+http://github.com/pyglet/pyglet@pyglet-1.5-maintenance
!pip install pygame

In [1]:
import GPUtil

gpus = GPUtil.getGPUs()
for gpu in gpus:
    print("GPU ID: {}, GPU Name: {}".format(gpu.id, gpu.name))
    print("GPU Load: {}, GPU Free Memory: {}".format(gpu.load, gpu.memoryFree))
    print("GPU Total Memory: {}, GPU Temperature: {}".format(gpu.memoryTotal, gpu.temperature))

GPU ID: 0, GPU Name: NVIDIA GeForce RTX 3060 Laptop GPU
GPU Load: 0.0, GPU Free Memory: 5996.0
GPU Total Memory: 6144.0, GPU Temperature: 57.0


In [ ]:
# 95eda70b703e651a93829d62187e30b7bc107306

import wandb
import gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.callbacks import CheckpointCallback

from wandb.integration.sb3 import WandbCallback

from huggingface_sb3 import load_from_hub, push_to_hub



config = {
    "env_name": "PongNoFrameskip-v4",
    "num_envs": 8,
    "total_timesteps": int(10e6),
    "seed": 4089164106,    
}

run = wandb.init(
    project="HFxSB3",
    config = config,
    sync_tensorboard = True,  # Auto-upload sb3's tensorboard metrics
    monitor_gym = True, # Auto-upload the videos of agents playing the game
    save_code = True, # Save the code to W&B
    )

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=8 => 8 environments)
env = make_atari_env(config["env_name"], n_envs=config["num_envs"], seed=config["seed"]) #PongNoFrameskip-v4

print("ENV ACTION SPACE: ", env.action_space.n)

# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)
# Video recorder
env = VecVideoRecorder(env, "videos", record_video_trigger=lambda x: x % 100000 == 0, video_length=2000)

# https://github.com/DLR-RM/rl-trained-agents/blob/10a9c31e806820d59b20d8b85ca67090338ea912/ppo/PongNoFrameskip-v4_1/PongNoFrameskip-v4/config.yml
model = PPO(policy = "CnnPolicy",
            env = env,
            batch_size = 256,
            clip_range = 0.1,
            ent_coef = 0.01,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 2.5e-4,
            max_grad_norm = 0.5,
            n_epochs = 4,
            n_steps = 128,
            vf_coef = 0.5,
            tensorboard_log = f"runs",
            verbose=1,
            )
    
model.learn(
    total_timesteps = config["total_timesteps"],
    callback = [
        WandbCallback(
        gradient_save_freq = 1000,
        model_save_path = f"models/{run.id}",
        ), 
        CheckpointCallback(save_freq=10000, save_path='./pong',
                                         name_prefix=config["env_name"]),
        ]
)

model.save("ppo-PongNoFrameskip-v4.zip")
push_to_hub(repo_id="ThomasSimonini/ppo-PongNoFrameskip-v4", 
    filename="ppo-PongNoFrameskip-v4.zip",
    commit_message="Added Pong trained agent")


In [4]:

# Import the libraries
import os 

import gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

from huggingface_sb3 import load_from_hub, push_to_hub

# Load the model
checkpoint = load_from_hub("ThomasSimonini/ppo-PongNoFrameskip-v4", "ppo-PongNoFrameskip-v4.zip")

# Because we using 3.7 on Colab and this agent was trained with 3.8 to avoid Pickle errors:
custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
        }

#model = PPO.load(checkpoint, custom_objects=custom_objects)

model = PPO.load(r"D:\Projects\Clg\RL\Agents\Pong\models\0r8nblin\model.zip")

env = make_atari_env('PongNoFrameskip-v4', n_envs=1)
env = VecFrameStack(env, n_stack=4)
env.metadata['render_fps'] = 60

d:\Envs\cuda_11_8\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(
d:\Envs\cuda_11_8\lib\site-packages\stable_baselines3\common\base_class.py:751: UserWarning: You are probably loading a model saved with SB3 < 1.7.0, we deactivated exact_match so you can save the model again to avoid issues in the future (see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). Original error: Error(s) in loading state_dict for ActorCriticCnnPolicy:
	Missing key(s) in state_dict: "pi_features_extractor.cnn.0.weight", "pi_features_extractor.cnn.0.bias", "pi_features_extractor.cnn.2.weight", "pi_features_extractor.cnn.2.bias", "pi_features_extractor.cnn.4.weight", "pi_features_extractor.cnn.4.bias", "pi_features_extractor.linear.0.weight", "pi_features_extractor.linear.0.bias", "vf_features_extractor.cnn.0.we

In [2]:
import gym
from gym.utils.play import play
import pygame
import numpy as np

mapping = {(pygame.K_UP,): 2, (pygame.K_DOWN,): 3}  # pong

env = gym.make('PongNoFrameskip-v4', render_mode='rgb_array')  # Use the newer version and set render_mode to rgb_array

# Utilize the play function to start the environment
play(env, zoom=3, fps=30,  keys_to_action=mapping)

# Remember to close the environment when done
env.close()

In [5]:
import gym
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
import pygame


pygame.init()
width, height = 210, 160  # Adjust based on your environment's frame size
zoom_factor = 4
screen = pygame.display.set_mode((width * zoom_factor, height * zoom_factor))

obs = env.reset()
clock = pygame.time.Clock()

score = 0
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, info = env.step(action)
    
    # Render the environment's state to the Pygame window
    frame = env.render(mode='rgb_array')
    frame = pygame.surfarray.make_surface(frame)
    frame = pygame.transform.rotate(frame, -90) 
    frame = pygame.transform.flip(frame, True, False)
    screen.blit(pygame.transform.scale(frame, (width * zoom_factor, height * zoom_factor)), (0, 0))
    pygame.display.flip()
    
    # Handle Pygame events
    for event in pygame.event.get():
           if event.type == pygame.KEYDOWN:
               if event.key == pygame.K_q:
                   done = True  # Exit if 'q' is pressed
                   break
    # Control the frame rate
    clock.tick(60)

    score += rewards
# Clean up
pygame.quit()
env.close()

d:\Envs\cuda_11_8\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [ ]:
%matplotlib inline


import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

obs = env.reset()
plt.figure(figsize=(5, 4))
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    img = env.render(mode='rgb_array')
    
    plt.imshow(img)
    plt.axis('off')
    
    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())
    
    if dones:
        break

ipythondisplay.clear_output(wait=True)
env.close()